# WEEKLY CORN PRICE

## INIT

In [30]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
df = pd.read_csv("./data/corn2013-2017.txt", names=["date", "price"], header=None)
df.head()

,date,price
0,2013-01-06,7.794975
1,2013-01-13,7.863400
2,2013-01-20,8.234920
3,2013-01-27,8.186260
4,2013-02-03,8.317480


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    248 non-null    object 
 1   price   248 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.0+ KB


In [4]:
df.describe()

,price
count,248.000000
mean,5.138566
std,1.295431
min,3.507000
25%,4.034125
50%,4.680250
75%,5.994695
max,8.317480


## STATE

In [5]:
df["market_state"] = (df.price - df.price.shift(1))
df.market_state = df.market_state.fillna(0)

df.head()

,date,price,market_state
0,2013-01-06,7.794975,0.000000
1,2013-01-13,7.863400,0.068425
2,2013-01-20,8.234920,0.371520
3,2013-01-27,8.186260,-0.048660
4,2013-02-03,8.317480,0.131220


In [6]:
df.market_state.dtype

dtype('float64')

In [7]:
df.market_state.head()

0    0.000000
1    0.068425
2    0.371520
3   -0.048660
4    0.131220
Name: market_state, dtype: float64

In [8]:
def set_state(diff: int) -> str:
    if diff > 0:
        return "INC"
    elif diff < 0:
        return "DEC"
    else:
        return "EQU"

In [9]:
df.market_state = df.market_state.apply(lambda x: set_state(x))

In [10]:
df.market_state.unique()

array(['EQU', 'INC', 'DEC'], dtype=object)

In [11]:
df = df[~(df.market_state == "EQU")]

In [12]:
df.market_state.value_counts()

DEC    132
INC    115
Name: market_state, dtype: int64

## TRANSITION MATRIX

Calculer la propabilité de :
- INC qui suit DEC
- DEC qui suit INC
- INC qui suit INC
- DEC qui suit DEC

In [33]:
le = preprocessing.LabelEncoder()
traj = le.fit_transform(df.market_state)

In [48]:
# with states labeled as successive integers starting with 0
# and returns a transition matrix, M,
# where M[i][j] is the probability of transitioning from i to j

def transition_matrix(transitions):
    n = 1 + max(transitions)  # number of states

    M = [[0] * n for _ in range(n)]

    for (i, j) in zip(transitions, transitions[1:]):
        # transitions = n & transitions[1:] = n+1
        M[i][j] += 1

    # now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

In [49]:
m = transition_matrix(traj)
for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))

0.58 0.42
0.48 0.52


1 = INC      
0 = DEC      
1st row and 1st column = DEC      
2nd row and 2nd column = INC         

58 % de chance qu'un DEC suit un DEC et 42 % pour que DEC soit suivi par un INC

## PREDICT WITH TRANSITION MATRIX

In [58]:
starting_state = traj[-1]
current_state = starting_state
 
state = list(le.inverse_transform(np.unique(traj)))
print(state[current_state], "--->", end=" ")

# Simulating a random walk on our Markov chain
# with 20 steps. Random walk simply means that
# we start with an arbitrary state and then we
# move along our markov chain.
n = 40
 
while n-1:
    # Deciding the next state using a random.choice()
    # function,that takes list of states and the probability
    # to go to the next states from our current state
    current_state = np.random.choice([0, 1], p=m[current_state])
     
    # printing the path of random walk
    print(state[current_state], "--->", end=" ")
    n -= 1
print("stop")

INC ---> DEC ---> INC ---> DEC ---> DEC ---> DEC ---> DEC ---> DEC ---> DEC ---> INC ---> DEC ---> INC ---> DEC ---> DEC ---> DEC ---> DEC ---> INC ---> INC ---> INC ---> INC ---> INC ---> DEC ---> DEC ---> INC ---> INC ---> INC ---> INC ---> INC ---> INC ---> INC ---> INC ---> DEC ---> INC ---> INC ---> DEC ---> INC ---> DEC ---> INC ---> INC ---> DEC ---> stop
